In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt


from scipy.optimize import curve_fit
from scipy.integrate import quad


# Constantes
c = 3e5  # velocidad de la luz en km/s

def distancia_cosmologica(z, H0, Omega_M, Omega_Lambda):
    """
    Calcula la distancia luminosa para los redshifts dados y parámetros cosmológicos.
    Usa la forma general para un universo plano \(\Lambda\)CDM.
    """
    def integrando(z):
        return 1.0 / np.sqrt(Omega_M * (1 + z)**3 + Omega_Lambda)

    integral, _ = quad(integrando, 0, z)
    return (c / H0) * (1 + z) * integral


# Esta versión de distancia_cosmologica acepta un array de valores z y devuelve un array de distancias
# excluded quiere decir que no se vectorizan los parámetros cosmológicos
distancia_cosmologica_vectorizada = np.vectorize(distancia_cosmologica, excluded=['H0', 'Omega_M', 'Omega_Lambda'])


def funcion_ajuste(z, H0, Omega_M):
    """
    Función de ajuste asumiendo un universo plano solo con materia y energía oscura, sin curvatura.
    Omega_Lambda se calcula como 1 - Omega_M para un universo plano.
    """
    Omega_Lambda = 1 - Omega_M
    return distancia_cosmologica_vectorizada(z, H0, Omega_M, Omega_Lambda)


# Leer la tabla de datos del SCP y ordenarla por redshift
data = np.genfromtxt('/content/drive/MyDrive/AJUSTES/SCPUnion2.1_mu_vs_z.txt')
datos_ordenados = sorted(data, key=lambda x: x[1])
zs, mm, dm = np.transpose(datos_ordenados)[1:4]

# Convertir magnitud a módulo de distancia y calcular distancia en Mpc
dMpc = 10 ** ((mm / 5) - 5)

# Ajustar los datos al modelo cosmológico
popt, pcov = curve_fit(lambda z, H0, Omega_M: funcion_ajuste(z, H0, Omega_M), zs, dMpc, p0=[70, 0.3])
H0_ajuste, Omega_M_ajuste = popt
Omega_Lambda_ajuste = 1 - Omega_M_ajuste

# Modelo de Einstein-de Sitter (sin energía oscura, Omega_M = 1)
popt_EdS, pcov_EdS = curve_fit(lambda z, H0: funcion_ajuste(z, H0, 1.0), zs, dMpc, p0=[70])
H0_EdS = popt_EdS[0]

# Graficar los datos y los ajustes
plt.figure(figsize=(10, 6))
plt.errorbar(zs, dMpc, yerr=(10 ** (dm / 5) - 1) * dMpc, fmt='o', label='Datos de Supernova', color='blue', markersize=5)
z_ajuste = np.linspace(0, max(zs), 500)
plt.plot(z_ajuste, [funcion_ajuste(z, H0_ajuste, Omega_M_ajuste) for z in z_ajuste], label=f'Ajuste \(\Lambda\)CDM: H0={H0_ajuste:.2f}, \(\Omega_M\)={Omega_M_ajuste:.2f}, \(\Omega_\Lambda\)={Omega_Lambda_ajuste:.2f}', color='green')
plt.plot(z_ajuste, [funcion_ajuste(z, H0_EdS, 1.0) for z in z_ajuste], label=f'Ajuste Einstein-de Sitter: H0={H0_EdS:.2f}', color='red')
plt.xlabel('Redshift (z)')
plt.ylabel('Distancia luminosa (Mpc)')
plt.title('Ajuste a Modelos Cosmológicos Demostrando Energía Oscura')
plt.legend()
plt.grid(True)
plt.show()

FileNotFoundError: /content/drive/MyDrive/AJUSTES/SCPUnion2.1_mu_vs_z.txt not found.

In [ ]:
def calcular_chi_cuadrado(observados, esperados, errores):
    return np.sum(((observados - esperados) / errores) ** 2)

def calcular_BIC(n, k, chi_cuadrado):
    return k * np.log(n) + n * np.log(chi_cuadrado / n)

# Datos
n = len(zs)  # Número de puntos de datos

# Calcula los errores en la distancia
errores_dMpc = (10 ** (dm / 5) - 1) * dMpc

# Calcula chi cuadrado para cada modelo
chi_cuadrado_LambdaCDM = calcular_chi_cuadrado(dMpc, funcion_ajuste(zs, *popt), errores_dMpc)
chi_cuadrado_EdS = calcular_chi_cuadrado(dMpc, funcion_ajuste(zs, H0_EdS, 1.0), errores_dMpc)

# Número de parámetros en cada modelo
k_LambdaCDM = 2  # H0 y Omega_M
k_EdS = 1        # Solo H0

# Calcula BIC para cada modelo
BIC_LambdaCDM = calcular_BIC(n, k_LambdaCDM, chi_cuadrado_LambdaCDM)
BIC_EdS = calcular_BIC(n, k_EdS, chi_cuadrado_EdS)

print(f'Chi-cuadrado para LambdaCDM: {chi_cuadrado_LambdaCDM}')
print(f'Chi-cuadrado para Einstein-de Sitter: {chi_cuadrado_EdS}')

print(f'BIC para LambdaCDM: {BIC_LambdaCDM}')
print(f'BIC para Einstein-de Sitter: {BIC_EdS}')


Chi-cuadrado para LambdaCDM: 685.911789345184
Chi-cuadrado para Einstein-de Sitter: 4748.527490171107
BIC para LambdaCDM: 110.00419500216132
BIC para Einstein-de Sitter: 1225.8488392514166


Se ve claramente como el ajuste es superior con la introducción de la energía oscura es necesaria para ajustar los datos. (El modelo de Einstein de Sitter no tiene energía oscura.)